## Getting YouTube captions

#### This notebook shows how to gather data from YouTube and create a dataframe with it.

In [85]:
#!pip install youtube_transcript_api

In [2]:
from googleapiclient.discovery import build
from youtube_transcript_api import YouTubeTranscriptApi
import pandas as pd
import datetime

In [3]:
api_key = API_KEY
# replace by the API key that you generate on Google Developers

video_ids = VIDEO_ID
# replace by the video id which is the part of the URL after 'https://www.youtube.com/watch?v='



In [4]:
youtube = build('youtube', 'v3', developerKey= api_key)
#create a YouTube Data API v.3 object

In [5]:
channel_ids = []
channel_titles = []
video_titles = []
transcriptions = []

In [77]:
#for video_id in video_ids:

srt = YouTubeTranscriptApi.get_transcript(video_id, languages = ['fr'])
# get the captions data
transcriptions = []
time_caption = []
for i in srt:
  transcriptions.append(i['text'])
  time = str(datetime.timedelta(seconds=round(i['start'])))[0:]
  time_caption.append(time)

request = youtube.videos().list(part= "contentDetails,snippet", id = video_ids[0])
response = request.execute()
#make a request to the data of the video and execute it
#the resulting response is a dictionary with lots of metadata of the video
channel_id = response['items'][0]['snippet']['channelId']
channel_id = [channel_id] *len(transcriptions)
channel_title = [response['items'][0]['snippet']['channelTitle']]* len(transcriptions)
video_id = [response['items'][0]['id']]* len(transcriptions)
video_title = [response['items'][0]['snippet']['title']]* len(transcriptions)

In [82]:
df_1 = pd.DataFrame({
                 
                 'video_title': video_title,
                 'video_id' : video_ids[0],
                 'channel_id': channel_id,
                 'channel_title': channel_title,
 #                'video_description': video_description,
 #                'upload_date' : upload_date,
 #                'duration': duration, 
                  'time_caption' : time_caption,
                  'transcript': transcriptions })
#create the dataframe